In [1]:
import _init_paths
import nengo_dl
import nengo
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import pickle

from configs.exp_configs import tf_exp_cfg as exp_cfg, nengo_dl_cfg as ngo_cfg
from utils.base_utils.data_prep_utils import get_batches_of_cifar_data
from utils.base_utils.exp_utils import collect_sim_data_spikes
from utils.nengo_dl_utils import get_nengo_dl_model
from utils.consts.dir_consts import EXP_OTPT_DIR
from utils.consts.exp_consts import SEED

/home/rgaurav/miniconda3/envs/nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:360: UserWarning: Layer '<class 'tensorflow.python.keras.layers.normalization_v2.BatchNormalization'>' already has a converter. Overwriting.
  "Layer '%s' already has a converter. Overwriting." % keras_layer


# Get the batched Test Data, the Nengo-DL Model

In [2]:
batches = get_batches_of_cifar_data(ngo_cfg, is_test=True)
nengo_model, nengo_probes_obj_lst = get_nengo_dl_model(
    (32, 32, 3), exp_cfg, ngo_cfg)

/home/rgaurav/miniconda3/envs/nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:555: UserWarning: Activation type <function softmax at 0x2b4c6ad46320> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation


In [3]:
print(exp_cfg)
print(ngo_cfg)

{'rf': 5e-05, 'batch_size': 64, 'epochs': 16, 'lr': 0.0001, 'nn_dlyr': 10, 'tf_model': {'name': 'model_1', 'layers': {'layer_1': Layer(num_kernels=32, kernel_dims=(3, 3), stride_dims=(1, 1)), 'layer_2': Layer(num_kernels=64, kernel_dims=(3, 3), stride_dims=(1, 1)), 'layer_3': Layer(num_kernels=64, kernel_dims=(3, 3), stride_dims=(2, 2)), 'layer_4': Layer(num_kernels=96, kernel_dims=(3, 3), stride_dims=(2, 2)), 'layer_5': Layer(num_kernels=128, kernel_dims=(3, 3), stride_dims=(2, 2))}}, 'tf_wts_otpt_dir': '/scratch/rgaurav//spiking_investigation/experiment_outputs//cifar10/model_1/tf_trained_wts/'}
{'tf_wts_inpt_dir': '/scratch/rgaurav//spiking_investigation/experiment_outputs//cifar10/model_1/tf_trained_wts/weights', 'ndl_res_otpt_dir': '/scratch/rgaurav//spiking_investigation/experiment_outputs//cifar10/model_1/ndl_relu_results/', 'spk_neuron': SpikingRectifiedLinear(), 'synapse': 0.005, 'sfr': 100, 'n_steps': 60, 'test_batch_size': 100, 'train_batch_size': 16}



# Create the Simulator and do inference

In [4]:
ndl_model_otpt = []
ndl_model_spks = []

with nengo_model.net:
  nengo_dl.configure_settings(stateful=False)

with nengo_dl.Simulator(
  nengo_model.net, minibatch_size=ngo_cfg["test_batch_size"], progress_bar=True, seed=SEED) as sim:
  nengo_input = nengo_probes_obj_lst[0]
  nengo_output = nengo_probes_obj_lst[-1]
  batch_num=1
  for batch in batches:
    sim_data = sim.predict_on_batch({nengo_input: batch[0]})
    for y_true, y_pred in zip(batch[1], sim_data[nengo_output]):
      ndl_model_otpt.append((np.argmax([y_true]), np.argmax(y_pred[-1]), y_pred))
    if batch_num == 1: # Collect only for one batch, else memory consumption is too high.
      mdl_spks = collect_sim_data_spikes(nengo_probes_obj_lst, sim_data)
      ndl_model_spks.extend(mdl_spks)
    print("Batch: %s Done." % batch_num)
    batch_num += 1
    if batch_num == 2:
      break

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:02                                               
Batch: 1 Done.


In [5]:
layers = list(nengo_model.layers.dict.values())[1:-1]
ens0 = layers[0].ensemble
tsi = 1 # Test Sample Index in first batch of size 100
print("MaxRates: ", sim.data[ens0].max_rates)
print("Layer Names: ", ndl_model_spks[0].keys()) # Take 0th sample i.e. the first test sample.

MaxRates:  [100. 100. 100. ... 100. 100. 100.]
Layer Names:  dict_keys(['conv2d.0', 'conv2d_1.0', 'conv2d_2.0', 'conv2d_3.0', 'conv2d_4.0', 'dense.0', 'dense_1.0'])


/home/rgaurav/miniconda3/envs/nengo-tf/lib/python3.7/site-packages/nengo_dl/simulator.py:2218: UserWarning: Checking parameters after simulator is closed; cannot fetch live values, so the initial values will be returned.
  "Checking parameters after simulator is closed; "


In [6]:
layer = "conv2d.0"
t_steps, n_neurons = ndl_model_spks[tsi][layer].shape
for i in range(n_neurons):
    spikes = np.round(ndl_model_spks[tsi][layer][:,i] * ngo_cfg["sfr"] * 1e-3)
    print(np.sum(spikes)*1000/t_steps)

166.66666666666666
0.0
0.0
0.0
50.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
16.666666666666668
116.66666666666667
0.0
100.0
50.0
0.0
0.0
0.0
16.666666666666668
0.0
16.666666666666668
233.33333333333334
0.0
116.66666666666667
50.0
50.0
0.0
0.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
16.666666666666668
83.33333333333333
0.0
0.0
0.0
100.0
50.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
233.33333333333334
0.0
100.0
50.0
50.0
0.0
0.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
16.666666666666668
83.33333333333333
0.0
0.0
0.0
100.0
50.0
0.0
133.33333333333334
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
233.33333333333334
0.0
100.0
50.0
50.0
0.0
0.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
16.666666666666668
83.33333333333333
0.0
0.0
0.0
100.0
50.0
0.0
133.33333333333334
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
233.33333333333334
0.0
100.0
50.0
50.0
0.0
0.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
16

0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
66.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
66.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
66.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
66.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
66.66666666666667
0.0
0

16.666666666666668
0.0
0.0
0.0
316.6666666666667
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
50.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
316.6666666666667
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
50.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
50.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
316.6666666666667
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
50.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
316.6666666666667
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
50.0
0.0
66.66666666666667
0.0
0.

0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
50.0
0.0
66.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.6666

0.0
0.0
0.0
0.0
0.0
0.0
0.0
200.0
0.0
0.0
66.66666666666667
16.666666666666668
0.0
0.0
50.0
33.333333333333336
0.0
0.0
50.0
0.0
0.0
66.66666666666667
283.3333333333333
0.0
50.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
316.6666666666667
0.0
0.0
83.33333333333333
50.0
0.0
0.0
133.33333333333334
0.0
0.0
0.0
0.0
0.0
0.0
50.0
283.3333333333333
0.0
50.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
316.6666666666667
0.0
0.0
83.33333333333333
50.0
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
66.66666666666667
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
300.0
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
50.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
316.6666666666667
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
300.0
0.0
50.0
0.0
50.0
0.0
0.0

...

0.0
0.0
0.0
100.0
83.33333333333333
0.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
166.66666666666666
50.0
0.0
66.66666666666667
0.0
66.66666666666667
0.0
100.0
50.0
0.0
0.0
0.0
50.0
66.66666666666667
0.0
0.0
0.0
0.0
83.33333333333333
83.33333333333333
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
166.66666666666666
33.333333333333336
0.0
50.0
0.0
66.66666666666667
16.666666666666668
100.0
66.66666666666667
0.0
0.0
0.0
50.0
66.66666666666667
0.0
16.666666666666668
0.0
0.0
83.33333333333333
83.33333333333333
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
150.0
33.333333333333336
0.0
50.0
0.0
66.66666666666667
16.666666666666668
100.0
66.66666666666667
0.0
0.0
0.0
50.0
66.66666666666667
0.0
16.666666666666668
0.0
0.0
83.33333333333333
83.33333333333333
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
133.33333333333334
33.333333333333336
0.0
50.0
0.0
50.0
0.0
83.33333333333333
66.66666666666667
0.0
0.0
0.0
50.0

0.0
16.666666666666668
0.0
83.33333333333333
0.0
16.666666666666668
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
83.33333333333333
0.0
33.333333333333336
0.0
0.0
66.66666666666667
166.66666666666666
0.0
0.0
150.0
0.0
0.0
0.0
133.33333333333334
100.0
66.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
66.66666666666667
0.0
33.333333333333336
33.333333333333336
0.0
0.0
0.0
0.0
33.333333333333336
83.33333333333333
0.0
50.0
0.0
0.0
0.0
150.0
16.666666666666668
0.0
66.66666666666667
0.0
0.0
0.0
0.0
133.33333333333334
83.33333333333333
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
116.66666666666667
0.0
0.0
83.33333333333333
50.0
0.0
0.0
0.0
166.66666666666666
100.0
0.0
16.666666666666668
0.0
0.0
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
83.33333333333333
0.0
0.0
0.0
0.0
16.666666666666668
16.666666666666668
16.666666666666668
116.66666666666667
0.0
0.0
100.0
66.66666666666667
16.666666666666668
0.0
0.0
183.33333333333334
116.66666666666667
0.0
16.6666666666666

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
0.0
0.0
16.666666666666668
16.666666666666668
0.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
100.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
116.66666666666667
0.0
0.0
33.333333333333336
33.333333333333336
0.0
0.0
33.333333333333336
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
150.0
0.0
33.333333333333336
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
166.66666666666666
0.0
0.0
33.333333333333336
50.0
0.0
0.0
66.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
116.66666666666667
16.666666666666668
33.333333333333336
0.0
66.66666666666667
0.0
0.0
0.0
83.33333333333333
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
116.66666666666667
0.0
0.0
0.0
0.0
0.0
16.666666666666668
83.33333333333333
16.666666666666668
66.66666666666667
0.0
0.0
0.0
0.0
16.666666666666668
0.0
50.0
0.0
33.33333

0.0
0.0
0.0
66.66666666666667
0.0
0.0
0.0
0.0
0.0
33.333333333333336
116.66666666666667
100.0
0.0
0.0
50.0
0.0
0.0
0.0
66.66666666666667
50.0
0.0
0.0
50.0
0.0
0.0
33.333333333333336
0.0
0.0
66.66666666666667
0.0
16.666666666666668
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
0.0
0.0
16.666666666666668
83.33333333333333
50.0
0.0
0.0
50.0
0.0
0.0
0.0
66.66666666666667
50.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
0.0
0.0
66.66666666666667
0.0
16.666666666666668
16.666666666666668
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
50.0
16.666666666666668
0.0
0.0
33.333333333333336
0.0
0.0
0.0
50.0
16.666666666666668
0.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
50.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
16.666

33.333333333333336
0.0
66.66666666666667
50.0
0.0
0.0
83.33333333333333
0.0
0.0
0.0
0.0
0.0
50.0
33.333333333333336
16.666666666666668
0.0
0.0
0.0
0.0
133.33333333333334
0.0
0.0
50.0
0.0
16.666666666666668
83.33333333333333
166.66666666666666
133.33333333333334
0.0
33.333333333333336
50.0
16.666666666666668
33.333333333333336
0.0
66.66666666666667
50.0
0.0
0.0
83.33333333333333
0.0
0.0
0.0
0.0
0.0
50.0
33.333333333333336
16.666666666666668
0.0
0.0
0.0
0.0
133.33333333333334
0.0
0.0
50.0
0.0
0.0
83.33333333333333
166.66666666666666
133.33333333333334
0.0
33.333333333333336
50.0
16.666666666666668
33.333333333333336
0.0
66.66666666666667
50.0
0.0
0.0
83.33333333333333
0.0
0.0
0.0
0.0
0.0
50.0
33.333333333333336
16.666666666666668
0.0
0.0
0.0
0.0
133.33333333333334
0.0
0.0
50.0
0.0
16.666666666666668
83.33333333333333
166.66666666666666
133.33333333333334
0.0
16.666666666666668
50.0
16.666666666666668
33.333333333333336
0.0
66.66666666666667
50.0
0.0
0.0
83.33333333333333
0.0
0.0
0.0
0.0


In [8]:
for rate in sim.data[ens0].max_rates:
  print(rate)

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

...

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

In [11]:
layer = "conv2d.0"
t_steps, n_neurons = ndl_model_spks[tsi][layer].shape
for i in range(n_neurons):
    spikes = np.round(ndl_model_spks[tsi][layer][:,i] * ngo_cfg["sfr"] * 1e-3)
    print(np.sum(spikes)*1000/t_steps)

333.3333333333333
0.0
0.0
0.0
100.0
0.0
0.0
0.0
50.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
216.66666666666666
0.0
0.0
0.0
50.0
250.0
0.0
216.66666666666666
116.66666666666667
0.0
0.0
16.666666666666668
33.333333333333336
0.0
33.333333333333336
483.3333333333333
0.0
233.33333333333334
116.66666666666667
116.66666666666667
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
50.0
183.33333333333334
0.0
0.0
0.0
200.0
100.0
0.0
300.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
83.33333333333333
483.3333333333333
0.0
216.66666666666666
100.0
116.66666666666667
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
50.0
50.0
183.33333333333334
0.0
0.0
0.0
200.0
100.0
0.0
283.3333333333333
0.0
0.0
0.0
0.0
16.666666666666668
0.0
83.33333333333333
483.3333333333333
0.0
216.66666666666666
100.0
116.66666666666667
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
50.0
50.0
183.33333333333334
0.0
0.0
0.0
200.0
100.0
0.0
283.3333333333333
0.0
0.0
0.0
0.0
16.666666666666668
0.0
83.33333333333333
483.3333333333333
0.0
216.66

133.33333333333334
600.0
0.0
133.33333333333334
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
616.6666666666666
0.0
0.0
133.33333333333334
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
600.0
0.0
133.33333333333334
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
616.6666666666666
0.0
0.0
133.33333333333334
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
600.0
0.0
133.33333333333334
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
616.6666666666666
0.0
0.0
133.33333333333334
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
616.6666666666666
0.0
133.33333333333334
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
616.6666666666666
0.0
0.0
133.33333333333334
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
616.6666666666666
0.0
133.33333333333334
0.0
133.33333333333334
0.0
0.0
0.0
0

33.333333333333336
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
16.666666666666668
0.0
583.3333333333334
0.0
0.0
150.0
133.33333333333334
0.0
66.66666666666667
233.33333333333334
0.0
0.0
0.0
0.0
0.0
0.0
83.33333333333333
566.6666666666666
0.0
100.0
0.0
116.66666666666667
0.0
0.0
33.333333333333336
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
633.3333333333334
0.0
0.0
166.66666666666666
116.66666666666667
0.0
0.0
250.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
116.66666666666667
583.3333333333334
16.666666666666668
116.66666666666667
0.0
100.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
600.0
0.0
0.0
150.0
133.33333333333334
0.0
0.0
250.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
100.0
600.0
0.0
100.0
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
633.3333333333334
0.0
0.0
150.0
133.33333333333334
0.0
0.0
283.3333333333333
0.0
16.666666666666668
0.0
0.0
0.0
0.0
116.66666666666667
600.0
0.0
116.66666666666667
0.0
116.

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
633.3333333333334
0.0
0.0
150.0
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
616.6666666666666
0.0
133.33333333333334
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
633.3333333333334
0.0
0.0
150.0
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
600.0
0.0
133.33333333333334
0.0
133.33333333333334
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
616.6666666666666
0.0
0.0
150.0
150.0
0.0
0.0
300.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
133.33333333333334
600.0
0.0
133.33333333333334
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
616.6666666666666
0.0
0.0
150.0
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
600.0
0.0
116.66666666666667
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
616.6666666666666
0.0
0.0
150.0
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.

0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
616.6666666666666
0.0
133.33333333333334
0.0
116.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
633.3333333333334
0.0
0.0
150.0
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
616.6666666666666
0.0
133.33333333333334
0.0
133.33333333333334
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
633.3333333333334
0.0
0.0
150.0
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
616.6666666666666
0.0
133.33333333333334
0.0
133.33333333333334
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
633.3333333333334
0.0
0.0
150.0
150.0
0.0
0.0
300.0
0.0
0.0
0.0
0.0
0.0
0.0
133.33333333333334
450.0
100.0
133.33333333333334
0.0
283.3333333333333
0.0
0.0
0.0
266.6666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
116.66666666666667
416.6666666666667
0.0
0.0
0.0
0.0
0.0
50.0
300.0
66.66666666666667
283.3333333333333
0.0
0.0
0.0
0.0
83.33333333333333
400.0
0.0
0.0
0.0
0.0
0.0
0.

...

50.0
50.0
0.0
0.0
133.33333333333334
0.0
0.0
0.0
0.0
0.0
0.0
83.33333333333333
50.0
0.0
0.0
0.0
0.0
266.6666666666667
0.0
16.666666666666668
83.33333333333333
0.0
16.666666666666668
150.0
233.33333333333334
333.3333333333333
83.33333333333333
100.0
33.333333333333336
83.33333333333333
83.33333333333333
0.0
33.333333333333336
50.0
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
33.333333333333336
0.0
0.0
0.0
0.0
250.0
0.0
0.0
66.66666666666667
0.0
16.666666666666668
133.33333333333334
233.33333333333334
316.6666666666667
50.0
83.33333333333333
16.666666666666668
83.33333333333333
66.66666666666667
0.0
16.666666666666668
33.333333333333336
0.0
16.666666666666668
166.66666666666666
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
16.666666666666668
0.0
0.0
0.0
16.666666666666668
233.33333333333334
0.0
0.0
50.0
0.0
33.333333333333336
83.33333333333333
216.66666666666666
250.0
16.666666666666668
66.66666666666667
16.666666666666668
100.0
66.66666666666667
0.0
33.333333333333336
0.0
0.0
50.

0.0
333.3333333333333
133.33333333333334
50.0
0.0
16.666666666666668
0.0
0.0
0.0
0.0
133.33333333333334
33.333333333333336
33.333333333333336
216.66666666666666
0.0
0.0
0.0
0.0
0.0
0.0
216.66666666666666
0.0
83.33333333333333
0.0
0.0
66.66666666666667
250.0
16.666666666666668
0.0
266.6666666666667
0.0
0.0
0.0
216.66666666666666
183.33333333333334
150.0
0.0
0.0
66.66666666666667
0.0
0.0
0.0
0.0
150.0
0.0
116.66666666666667
33.333333333333336
83.33333333333333
0.0
0.0
0.0
216.66666666666666
166.66666666666666
0.0
100.0
83.33333333333333
0.0
0.0
150.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
0.0
83.33333333333333
83.33333333333333
0.0
33.333333333333336
0.0
0.0
83.33333333333333
0.0
0.0
66.66666666666667
0.0
33.333333333333336
16.666666666666668
233.33333333333334
16.666666666666668
0.0
0.0
366.6666666666667
150.0
16.666666666666668
66.66666666666667
83.33333333333333
0.0
0.0
66.66666666666667
0.0
0.0
0.0
0.0
33.333333333333336
16.666666666666668
0.0
0.0
0.0
133.33333333333334
16.6666666666

216.66666666666666
16.666666666666668
50.0
0.0
233.33333333333334
183.33333333333334
0.0
0.0
100.0
0.0
0.0
50.0
0.0
150.0
16.666666666666668
50.0
300.0
0.0
0.0
0.0
0.0
100.0
0.0
183.33333333333334
216.66666666666666
50.0
0.0
233.33333333333334
216.66666666666666
383.3333333333333
83.33333333333333
0.0
216.66666666666666
0.0
50.0
0.0
233.33333333333334
183.33333333333334
0.0
0.0
100.0
0.0
0.0
50.0
0.0
150.0
33.333333333333336
50.0
300.0
0.0
0.0
0.0
0.0
100.0
0.0
200.0
216.66666666666666
50.0
0.0
233.33333333333334
200.0
383.3333333333333
100.0
0.0
200.0
0.0
50.0
0.0
216.66666666666666
183.33333333333334
0.0
0.0
83.33333333333333
0.0
0.0
50.0
0.0
166.66666666666666
33.333333333333336
50.0
300.0
0.0
0.0
0.0
0.0
116.66666666666667
0.0
200.0
216.66666666666666
33.333333333333336
0.0
233.33333333333334
200.0
383.3333333333333
100.0
0.0
200.0
0.0
66.66666666666667
0.0
216.66666666666666
166.66666666666666
0.0
0.0
100.0
0.0
0.0
50.0
0.0
166.66666666666666
16.666666666666668
50.0
300.0
0.0
0.0


0.0
16.666666666666668
0.0
166.66666666666666
0.0
0.0
50.0
50.0
16.666666666666668
0.0
50.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
33.333333333333336
216.66666666666666
0.0
33.333333333333336
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33.333333333333336
0.0
16.666666666666668
0.0
266.6666666666667
0.0
0.0
66.66666666666667
66.66666666666667
0.0
0.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
50.0
300.0
0.0
66.66666666666667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
50.0
0.0
16.666666666666668
0.0
350.0
0.0
0.0
100.0
100.0
0.0
0.0
150.0
0.0
0.0
0.0
0.0
0.0
0.0
66.66666666666667
250.0
33.333333333333336
83.33333333333333
0.0
150.0
0.0
0.0
0.0
183.33333333333334
0.0
0.0
0.0
0.0
0.0
0.0
0.0
50.0
233.33333333333334
0.0
0.0
16.666666666666668
0.0
0.0
33.333333333333336
183.33333333333334
50.0
166.66666666666666
0.0
0.0
0.0
0.0
33.333333333333336
0.0
116.66666666666667
0.0
66.66666666666667
33.333333333333336
100.0
166.66666666666666
0.0
100.0
100.0
16.666666666666668
16.666666666666668
16.666666666666668
33.33

0.0
0.0
0.0
0.0
0.0
16.666666666666668
16.666666666666668
0.0
0.0
16.666666666666668
16.666666666666668
0.0
0.0
0.0
33.333333333333336
0.0
0.0
0.0
33.333333333333336
16.666666666666668
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
16.666666666666668
50.0
0.0
33.333333333333336
16.666666666666668
0.0
16.666666666666668
0.0
0.0
0.0
16.666666666666668
16.666666666666668
0.0
0.0
16.666666666666668
16.666666666666668
16.666666666666668
0.0
0.0
16.666666666666668
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
16.666666666666668
0.0
0.0
16.666666666666668
16.666666666666668
33.333333333333336
16.666666666666668
0.0
33.333333333333336
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
16.666666666666668
0.0
16.666666666666668
33.333333333333336
0.0
0.0
0.0
16.666666666666668
0.0
0.0
0.0
33.333333333333336
0.0
0.0
16.666666666666668
0.0
0.0
16.666666666666668
0.0
16.666666666666668
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.666666666666668
0.0
0.0
16.666666666666668
33.333333333333336


## Inference with Encoders, Bias, Gain, Plots, etc. for 100th batch

ndl_model_otpt = []
ndl_model_spks = []

with nengo_model.net:
  nengo_dl.configure_settings(stateful=False)

with nengo_dl.Simulator(
  nengo_model.net, minibatch_size=ngo_cfg["test_batch_size"], progress_bar=True, seed=SEED) as sim:
  nengo_input = nengo_probes_obj_lst[0]
  nengo_output = nengo_probes_obj_lst[-1]
  batch_num=1
  for batch in batches:
    sim_data = sim.predict_on_batch({nengo_input: batch[0]})
    for y_true, y_pred in zip(batch[1], sim_data[nengo_output]):
      ndl_model_otpt.append((np.argmax([y_true]), np.argmax(y_pred[-1]), y_pred))
    if batch_num == 1: # Collect only for one batch, else memory consumption is too high.
      mdl_spks = collect_sim_data_spikes(nengo_probes_obj_lst, sim_data)
      ndl_model_spks.extend(mdl_spks)
    print("Batch: %s Done." % batch_num)
    batch_num += 1
    if batch_num == 100:
        layers = list(nengo_model.layers.dict.values())[1:-1]
        fig, axs = plt.subplots(len(layers), figsize=(16, 3*len(layers)))
        fig1, axs1 = plt.subplots(len(layers), figsize=(16, 3*len(layers)))
        for i, layer in enumerate(layers):
            ens = layer.ensemble
            axs[i].set_title("Layer: {0}, Neuron Type: {1}, Number of Neurons: {2}, Seed: {3}".format(
                ens.neurons, ens.neuron_type, ens.n_neurons, ens.seed))
            axs[i].plot(sim.data[ens].max_rates, 'ro', label="Max Firing Rates")
            print("Layer: {0}, Neuron Type: {1}, Number of Neurons: {2}, Seed: {3}".format(
                ens.neurons, ens.neuron_type, ens.n_neurons, ens.seed))
            print("Unique Encoders: {}".format(np.unique(sim.data[ens].encoders)))
            print("Unique Scaled Encoders: {}".format(np.unique(sim.data[ens].scaled_encoders)))
            print("Unique bias: {}".format(np.unique(sim.data[ens].bias)))
            print("Unique gain: {}".format(np.unique(sim.data[ens].gain)))
            print("Unique intercepts: {}".format(np.unique(sim.data[ens].intercepts)))
            axs[i].legend()
            axs[i].set_xlabel("Neuron Indices")
            print("*"*100)
            axs1[i].set_title("Tuning Curves for {}".format(ens.neurons))
            x, act_mat = nengo.utils.ensemble.tuning_curves(ens, sim)
            for j in range(ens.n_neurons):
                axs1[i].plot(act_mat[:, j])
            axs1[i].set_xlabel("x - values")
        fig.tight_layout()
        fig.show()
        fig1.tight_layout()
        fig1.show()

In [6]:
pickle.dump({"acc_res": ndl_model_otpt, "spk_res": ndl_model_spks}, 
            open(ngo_cfg["ndl_res_otpt_dir"]+"ndl_%s_results_sfr_%s_nstps_%s.p" 
                % (exp_cfg["tf_model"]["name"], ngo_cfg["sfr"], ngo_cfg["n_steps"]), "wb"))

In [6]:
acc = 0
for y_true, y_pred, _ in ndl_model_otpt:
  if y_true == y_pred:
    acc += 1

print(acc/10000)

0.6256


In [5]:
# SFR = 1, N_STEPS=80, ACC: 10% random (No layer spikes)
# SFR = 50, N_STEPS=80, ACC: 42.1 (output layer starts spiking after 70ms)
# SFR = 100, N_STEPS=80, ACC: 60.73 (output layer starts spiking after 50ms)
# SFR = 200, N_STEPS=80, ACC: 63.85 (output layer starts spiking after 35ms)
# SFR = 250, N_STEPS=80, ACC: 64.56 (output layer starts spiking after 30ms)
# SFR = 600, N_STEPS=80, ACC: 64.98 (output layer starts spiking after 20ms)


# SFR = 250, N_STEPS=60, ACC: 63.85 (Same spiking pattern as N_STEPS=80)
# SFR = 250, N_STEPS=55, ACC: 62.21 (Same spiking pattern as N_STEPS=80)
# SFR = 250, N_STEPS=50, ACC: 60.02 (Same spiking pattern as N_STEPS=80)
# SFR = 250, N_STEPS=40, ACC: 40.87 (Same spiking pattern as N_STEPS=80)


# As you lower down the SFR (and keep the N_STEPS fixed), it takes longer for the output layer to spike.
# 

acc = 0
for y_true, y_pred, _ in ndl_model_otpt:
  if y_true == y_pred:
    acc += 1

print(acc/10000)

0.6383
